In [13]:
import sys
import re
assert sys.version_info >= (3, 5) # make sure we have Python 3.5+
from pyspark.sql import SparkSession, types
from pyspark.sql.functions import udf, concat_ws
spark = SparkSession.builder.appName('entity resolution').getOrCreate()
spark.sparkContext.setLogLevel('WARN')
sc = spark.sparkContext
assert spark.version >= '2.3' # make sure we have Spark 2.3+

In [16]:
class EntityResolution:
    def __init__(self, dataFile1, dataFile2, stopWordsFile):
        self.f = open(stopWordsFile, "r")
        self.stopWords = set(self.f.read().split("\n"))
        self.stopWordsBC = sc.broadcast(self.stopWords).value
        self.df1 = spark.read.parquet(dataFile1).cache()
        self.df2 = spark.read.parquet(dataFile2).cache()
    

    def preprocessDF(self, df, cols): 
        df = df.withColumn("new_col",concat_ws(' ', df[cols[0]], df[cols[1]]))
        #df2 = df1.withColumn("col2", functions.lower(df1.col1))
        splitter = udf(lambda x: re.split(r'\W+', x), types.ArrayType(types.StringType()))
        df = df.withColumn("new_col", splitter('new_col'))
        df.show()
        stopWords = self.stopWordsBC
        #stop_words_remover = udf(lambda x: [item for item in x if item not in stopWords], ArrayType(StringType()))
        #stop_words_df = token_df.withColumn("joinKey", stop_words_remover(token_df.token_cols))

        #return stop_words_df.drop("concat_col").drop("lower_col").drop("token_cols")
        return df
    def filtering(self, df1, df2):
        """
            Write your code!
        """

    def verification(self, candDF, threshold):
        """
            Write your code!
        """

    def evaluate(self, result, groundTruth):
        """
            Write your code!
        """

    def jaccardJoin(self, cols1, cols2, threshold):
        newDF1 = self.preprocessDF(self.df1, cols1)
        newDF2 = self.preprocessDF(self.df2, cols2)
        print ("Before filtering: %d pairs in total" %(self.df1.count()*self.df2.count())) 
        newDF1.select('new_col').show(20,False)
        newDF2.select('new_col').show(20,False)

        #candDF = self.filtering(newDF1, newDF2)
        #print ("After Filtering: %d pairs left" %(candDF.count()))

        #resultDF = self.verification(candDF, threshold)
        #print ("After Verification: %d similar pairs" %(resultDF.count()))

        #return resultDF


    def __del__(self):
        self.f.close()


In [17]:
amazon_input = 'Amazon_sample'
google_input = 'Google_sample'
stopwords_input = 'stopwords.txt'
perfect_mapping_input = 'Amazon_Google_perfectMapping_sample'
er = EntityResolution(amazon_input, google_input, stopwords_input)
amazonCols = ["title", "manufacturer"]
googleCols = ["name", "manufacturer"]
resultDF = er.jaccardJoin(amazonCols, googleCols, 0.5)


Py4JJavaError: An error occurred while calling o462.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 21.0 failed 1 times, most recent failure: Lost task 0.0 in stage 21.0 (TID 21, localhost, executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:170)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:97)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:108)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:77)
	at org.apache.spark.sql.execution.python.EvalPythonExec$$anonfun$doExecute$1.apply(EvalPythonExec.scala:127)
	at org.apache.spark.sql.execution.python.EvalPythonExec$$anonfun$doExecute$1.apply(EvalPythonExec.scala:89)
	at